<a href="https://colab.research.google.com/github/Dilru1/X-ray_Files/blob/main/X_ray_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np 
import sys
from scipy import special


## Mount Drive

Before mounting the drive click on [this folder](https://drive.google.com/drive/folders/1rXKMGDayWkZCsbHAcIANkZhYYTdqpCjx?usp=sharing) and add it to your google drive by following these steps:

*   Go to your drive 
*   Find shared folder ("Shared with me" link)
*   Right click it
*   "Add Shortcut to Drive"

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
def init():
    # Root - working folder
    global mypath
    mypath ='/content/drive/My Drive/X-ray_project_Data/data/'
    
    # Folder conatining the code and the observation list
    global flux,maps
    flux = mypath+'flux_eff/'
    maps = mypath+ 'maps_eff/'
init()


In [14]:
#data file for each epoch 
ima_filename1 = np.array(fits.open(maps+'mosa-6320-6480_sub_30arcsec_2000.fits')[0].data) #2000
ima_filename2 = np.array(fits.open(maps+'mosa-6320-6480_sub_30arcsec_2004.fits')[0].data) #2004
ima_filename3 = np.array(fits.open(maps+'mosa-6320-6480_sub_30arcsec_2012.fits')[0].data) #2008
ima_filename4 = np.array(fits.open(maps+'mosa-6320-6480_sub_30arcsec_2018.fits')[0].data) #2012
ima_filename5 = np.array(fits.open(maps+'mosa-6320-6480_sub_30arcsec_2020.fits')[0].data) #2018

#error file for each epoch 

ima_netfilename1 = np.array(fits.open(maps+'Dmosa-6320-6480_sub_30arcsec_2000.fits')[0].data)
ima_netfilename2 = np.array(fits.open(maps+'Dmosa-6320-6480_sub_30arcsec_2004.fits')[0].data)
ima_netfilename3 = np.array(fits.open(maps+'Dmosa-6320-6480_sub_30arcsec_2012.fits')[0].data)
ima_netfilename4 = np.array(fits.open(maps+'Dmosa-6320-6480_sub_30arcsec_2018.fits')[0].data)
ima_netfilename5 = np.array(fits.open(maps+'Dmosa-6320-6480_sub_30arcsec_2020.fits')[0].data)


In [16]:
directory = os.fsencode(maps)
myfiles=[]
for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if filename.endswith(".fits") and str(filename)[  (len(str(filename)) -9) : (len(str(filename)) -5) ] in ('2000','2004','2012','2018','2020' )   :
         myfiles.append(filename)
     else:
         continue

print(len(myfiles)," Files are located in data folder ")
myfiles.sort()

path = os.getcwd()

#Read in multiple files and store as multiple arrays  (See Ref :- https://stackoverflow.com/questions/57570095/how-to-read-in-multiple-files-and-store-as-multiple-arrays)
d={}
tup=0
years = (2000,2004,2012,2018,2020,2000,2004,2012,2018,2020)

for (i,j) in zip( range(len(myfiles)), years ):
    os.chdir(directory)
    filename = os.path.basename(myfiles[tup])
    s=fits.open(filename)[0].data
    paths =os.path.basename(''.join(myfiles[tup]))
    s=fits.open(filename)[0].data
    if i <= 4 :
        d['error_{0}'.format(j)]= np.array(s)
    if i > 4 :
        d['flux_{0}'.format(j)]= np.array(s)
    tup +=1
    
directory = os.fsencode('/content/drive/My Drive/X-ray_project_Data/Analy/')
os.chdir(directory)

10  Files are located in data folder 


In [17]:
def select_detect_data(d,nrow,ncol):
    flux=[]
    error=[]
    for key in d:
        if key in ('error_2000', 'error_2004', 'error_2012' , 'error_2018', 'error_2020'):
            error.append(d[key][nrow][ncol])
        else:
            flux.append(d[key][nrow][ncol])
    return flux,error

In [19]:
y_min= -4.5665634861108334e-07
y_max= 4.820102166526607e-06
N_grid=100
Grid= np.linspace(y_min, y_max , N_grid )
sqrt2= np.sqrt(2) 

In [20]:
def select_ftval(PIXEL_X,PIXEL_Y):
    sample_data  =np.zeros(5)
    sample_error =np.zeros(5)
    B=0
    P_new,P_old, ft_val=[],[], []
    for i in Grid:
        sample_data  =np.array( select_detect_data(d,PIXEL_X,PIXEL_Y)[0]  )
        sample_error =np.array( select_detect_data(d,PIXEL_X,PIXEL_Y)[1]  )
        sample_error=np.ma.masked_invalid(sample_error)
        sample_data= np.ma.masked_values(sample_data,0)
        xval= (i- sample_data )/( (sqrt2*sample_error) )
        errf= 0.5*special.erfc(xval)
        P= np.prod(errf)
        D= 2*(np.sum(1-P))
        Q_old = P**(1/5)
        Q_new =P**(1/D)
        P_new.append(Q_new)
        P_old.append(Q_old)
        ft_val.append(i)

    P_new=np.array(P_new)
    P_old=np.array(P_old)
    ft_val=np.array(ft_val)

    try:
        B = np.max( np.where(P_new > 0.5 )[0] )
        ft_map=ft_val[B]
        return ft_map
    except ValueError:  #raised if `y` is empty.
        pass
   

test=[]
map=np.zeros((30, 30))
for i in range(30):
    for j in range(30):
        test.append(select_ftval(i,j))
test= np.array(test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars


In [21]:
def select_ftval(PIXEL_X,PIXEL_Y):
    sample_data  =np.zeros(5)
    sample_error =np.zeros(5)
    B=0
    P_new,P_old, ft_val=[],[], []
    for i in Grid:
        sample_data  =np.array( select_detect_data(d,PIXEL_X,PIXEL_Y)[0]  )
        sample_error =np.array( select_detect_data(d,PIXEL_X,PIXEL_Y)[1]  )
        sample_error=np.ma.masked_invalid(sample_error)
        sample_data= np.ma.masked_values(sample_data,0)
        xval= (i- sample_data )/( (sqrt2*sample_error) )
        errf= 0.5*special.erfc(xval)
        P= np.prod(errf)
        D= 2*(np.sum(1-P))
        Q_old = P**(1/5)
        Q_new =P**(1/D)
        P_new.append(Q_new)
        P_old.append(Q_old)
        ft_val.append(i)

    P_new=np.array(P_new)
    P_old=np.array(P_old)
    ft_val=np.array(ft_val)

    #B = ( np.where(P_new > 0.5 )[0] )[-1]
    #ft_ee= ft_val[B]
    #print("last ft value is :",ft_ee)
    #eturn ft_ee

    try:
        B1 = np.max( np.where(P_old > 0.5 )[0] )
        ft_mapold=ft_val[B1]
        return ft_mapold
    except ValueError:  #raised if `y` is empty.
        pass
   

testold=[]
map=np.zeros((30, 30))
for i in range(30):
    for j in range(30):
        testold.append(select_ftval(i,j))
testold= np.array(testold)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()
